In [ ]:
import cv2
import numpy as np
from openvino.inference_engine import IENetwork
from openvino.inference_engine import IEPlugin

In [ ]:
# Methods for object detection

def setup_obj_net(precision, device_choice):
    # path_to_objxml = './gesture_ssd_fp16/frozen_inference_graph.xml'
    # path_to_objbin = './gesture_ssd_fp16/frozen_inference_graph.bin'
    
    path_to_objxml = './gesture_folders/test_fp16/frozen_inference_graph.xml'
    path_to_objbin = './gesture_folders/test_fp16/frozen_inference_graph.bin'
    
    if precision.lower() == 'fp32':
        # path_to_objxml = './gesture_ssd/frozen_inference_graph.xml'
        # path_to_objbin = './gesture_ssd/frozen_inference_graph.bin'
    
        path_to_objxml = './test_fp32/frozen_inference_graph.xml'
        path_to_objbin = './test_fp32/frozen_inference_graph.bin'
    else:
        print('lower precision')

    net = IENetwork(model=path_to_objxml, weights=path_to_objbin)
    input_layer = next(iter(net.inputs))
    output_layer = next(iter(net.outputs))
    input_shape = net.inputs[input_layer].shape
    
    obj_plugin = IEPlugin(device=device_choice)
    if device_choice.lower() == 'cpu':
        ext = '/opt/intel/openvino/deployment_tools/inference_engine/'
        ext = ext + 'lib/intel64/libcpu_extension_avx2.so'
        obj_plugin.add_cpu_extension(ext)
    obj_exec_net = obj_plugin.load(network=net, num_requests=1)
    
    return {
        'net': obj_exec_net, 
        'input_layer': input_layer,
        'output_layer': output_layer,
        'input_shape': input_shape
    }

def pre_obj_processing(obj_frame, input_shape):
    n, c, h, w = input_shape
    obj_in_frame = cv2.resize(obj_frame, (w, h))
    obj_in_frame = obj_in_frame.transpose((2, 0, 1))
    obj_in_frame = obj_in_frame.reshape((n, c, h, w))
    
    return {
        'blob' : obj_in_frame, 
        'frame': obj_frame, 
    }

def draw_bb(obj_det, frame):
    i_w = frame.shape[1]
    i_h = frame.shape[0]
    drawn = False
    
    proposals = []
    
    for obj in obj_det[0][0]:
        if obj[2] > 0.5:
            proposals.append(obj)
    if len(proposals) == 0:
        return {'status': False, 'miniframe': None, 'frame': frame, 'class': 8, 'offset': None}
    
    proposed_obj = max(proposals, key=lambda x: x[2])
    xmin = int(proposed_obj[3] * i_w)
    ymin = int(proposed_obj[4] * i_h)
    xmax = int(proposed_obj[5] * i_w)
    ymax = int(proposed_obj[6] * i_h)
    class_id = int(proposed_obj[1])
    green = (0, 255, 0)
    cv2.putText(frame, str(class_id), (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 0.8, green, 2, cv2.LINE_AA )
    cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), green, 2)
    
    xmid = (xmin + xmax) / 2
    ymid = (ymin + ymax) / 2
    
    xmin = int(xmid - 149)
    xmax = int(xmid + 150)
    ymin = int(ymid - 149)
    ymax = int(ymid + 150)
    
    if ymin >= i_h or xmin >= i_w or ymin < 0 or xmin < 0:
        return {'status': False, 'miniframe': None, 'frame': frame, 'class': 8, 'offset': None}
    
    # Draw box and label\class_id
    cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 255, 255), 2)
    
    return {'status': True, 'miniframe': frame[ymin:ymax, xmin:xmax], 
            'frame': frame, 'class': class_id, 'offset': (xmin, ymin)}

In [ ]:
# Obj Detection Demo
def main():
    # object detection net
    net_dict = setup_obj_net('fp16', 'MYRIAD')
    obj_exec_net = net_dict['net']
    input_shape = net_dict['input_shape']
    input_layer = net_dict['input_layer']
    output_layer = net_dict['output_layer']
    
    vs = cv2.VideoCapture(0)
    while True:
        ret, vframe = vs.read()
        image_dict = pre_obj_processing(vframe, input_shape)
        inpBlob = image_dict['blob']

        obj_res = obj_exec_net.infer({'image_tensor': inpBlob})
        obj_det = obj_res['DetectionOutput']
        
        draw_bb(obj_det, image_dict['frame'])
        cv2.imshow("Frame", image_dict['frame'])

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
 
    # do a bit of cleanup
    cv2.destroyAllWindows()

main()